# Running MoCHI on DTS01 (Combinatorial-1) and DTS05 (Combinatorial-2) datasets - *all variants*

1. Allowing up to 2nd order terms
2. Fitting the following functions: Sigmoid, ELU and Linear
3. `l2_regularization_factor` = 10^(-5)

`mochi_all_variants_DTS01_20240424.tsv` and likewise for DTS05

24.04.2024

In [1]:
import pymochi
from pymochi.data import MochiData
from pymochi.models import MochiTask
from pymochi.project import MochiProject
from pymochi.report import MochiReport
import pandas as pd
import numpy as np
import pickle

In [2]:
%%bash

pip freeze

asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
certifi @ file:///home/conda/feedstock_root/build_artifacts/certifi_1707022139797/work/certifi
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1710320294760/work
cycler @ file:///home/conda/feedstock_root/build_artifacts/cycler_1696677705766/work
debugpy @ file:///home/conda/feedstock_root/build_artifacts/debugpy_1707444393922/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
exceptiongroup @ file:///home/conda/feedstock_root/build_artifacts/exceptiongroup_1704921103267/work
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1698579936712/work
fonttools @ file:///home/conda/feedstock_root/build_artifacts/fonttools_1710865504921/work
importlib_metadata @ file:///home/conda/feedstock_root/build_artifacts/importlib-metadata_1709821103657/work
ipykernel @ file:///home/conda/feedstock_root/build_artifacts/ipykernel_170899

In [3]:
%%bash

#!/usr/bin/env

pwd


/lustre/scratch126/gengen/projects/amyloid_beta_epistasis/DTS_joint_analysis/selected_DTS_datasets_analysis/DTS01_DTS05


In [4]:
filedir = '/lustre/scratch126/gengen/projects/amyloid_beta_epistasis/DTS_joint_analysis/files/'

datasets = ['DTS01',
            'DTS05',
           ]

filenames = {}
for dataset in datasets:
    filenames[dataset] = 'mochi_all_variants_' + dataset + '_20240424.tsv'

output_dir = '/lustre/scratch126/gengen/projects/amyloid_beta_epistasis/DTS_joint_analysis/mochi_results/'

In [5]:
# For joint modelling making sure that the aa_seq is comlemented the full Ab sequence
# this was done in notebook M0

all_vars = {}

total_n_vars = 0

for dataset in datasets:
    print(dataset)
    all_vars[dataset] = pd.read_csv(filedir + filenames[dataset], sep='\t')
    # check length of aa_seq, should be already complemented to full Ab sequence (42 aa)
    print(np.unique([len(elem) for elem in all_vars[dataset]['aa_seq']], return_counts=True), '\n')
    
    total_n_vars = total_n_vars + len(all_vars[dataset])

DTS01
(array([42]), array([37671])) 

DTS05
(array([42]), array([5283])) 



In [6]:
# 213771 variants in total, 182099 without DTS02, 45925 for ensemble of DTS01+DTS05+DTS14
total_n_vars

42954

In [7]:
# actual WT of Ab, artificially added to these tables in notebook M0
wt_vars = {}

for dataset in datasets:
    print(dataset)
    print(all_vars[dataset][all_vars[dataset]['WT'] == 1])
    wt_vars[dataset] = list(all_vars[dataset][all_vars[dataset]['WT'] == 1]['aa_seq'])[0]
    print('\n')

DTS01
                                           aa_seq  Nham_aa   WT  fitness  \
37670  DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIA        0  1.0      0.0   

       sigma  
37670  100.0  


DTS05
                                          aa_seq  Nham_aa   WT  fitness  sigma
5282  DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIA        0  1.0      0.0  100.0




In [8]:
wt_vars

{'DTS01': 'DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIA',
 'DTS05': 'DAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIA'}

In [9]:
# setting l2_regularization_factor to 10^(-5)
l2_regularization_factor_value = 0.00001

# Fitting a joint model first

In [ ]:
# started at 15:40 (24.04.2024)
# fold1 of model 1 began at X
# overall 2 models to fit (1 function fits x 2 N_max_interaction_order)
# with 64 Gb of RAM and 8 CPU cores
# using 80 Gb GPU RAM

k_folds = 10

transformations = ['Sigmoid', #'ELU', 'Linear'
                  ] 

N_max_interaction_order = [2, 1]

for transformation in transformations:
    print('Transformation:', transformation)
        
    for n_max_interaction_order in N_max_interaction_order:
        print('N_max_interaction_order', n_max_interaction_order)
    
        my_model_design = pd.DataFrame({
                   'phenotype': ['Nucleation_' + dataset for dataset in datasets],

                    # transformation can be one of: Linear, ReLU, SiLU, Sigmoid, SumOfSigmoids, 
                    #                               TwoStateFractionFolded, ThreeStateFractionFolded, FourStateFractionFolded

                   'transformation': [transformation for dataset in datasets], 
                   'trait': ['Nucleation' for dataset in datasets],
                   'file': [filedir + filenames[dataset] for dataset in datasets]})

        mochi_project = MochiTask(
                   directory = output_dir + '20240424_max_terms_order_' + str(n_max_interaction_order) + '_' + transformation + '_l2_regularization_factor_' + str(l2_regularization_factor_value) + '_DTS01_DT05_all_variants',
                   data = MochiData(
                      model_design = my_model_design,
                      max_interaction_order = n_max_interaction_order,
                      k_folds = k_folds),
                l2_regularization_factor = l2_regularization_factor_value,

                # this was 200 before by default - now might be a better fit - can try different values in the future
            sos_architecture = [5])

        # started at X
        # takes about 6-7?? minutes
        mochi_project.grid_search() 

        for i in range(k_folds):
            mochi_project.fit_best(fold = i+1)

        temperature = 30

        mochi_report = MochiReport(
                   task = mochi_project, # changed from project = mochi_project, there must have been a change in agrument name
                   RT = (273+temperature)*0.001987)

        energies = mochi_project.get_additive_trait_weights(
                   RT = (273+temperature)*0.001987)

        mochi_project.save()
            
        #print('Done with l2_regularization_factor_value', l2_regularization_factor_value, '\n', '################')

        print('Done with transformation', transformation, '\n', '################')
        
    print('Done with n_max_interaction_order', n_max_interaction_order, '\n', '################')

Transformation: Sigmoid
N_max_interaction_order 2
Loading fitness data
One-hot encoding sequence features
One-hot encoding interaction features
... Total theoretical features (order:count): 2:640
... Total retained features (order:count): 2:640 (100.0%)
Defining cross-validation groups
Defining coefficient groups
Done!
Performing grid search...
Fitting model:
{'fold': 1, 'seed': 1, 'grid_search': True, 'batch_size': 512, 'learn_rate': 0.05, 'num_epochs': 1000, 'num_epochs_grid': 100, 'l1_regularization_factor': 0.0, 'l2_regularization_factor': 1e-05, 'training_resample': True, 'early_stopping': True, 'scheduler_gamma': 0.98, 'scheduler_epochs': 10, 'loss_function_name': 'WeightedL1', 'sos_architecture': [5], 'sos_outputlinear': False}
Epoch 1; Avg_val_loss: 1.1218; WTcoef_1: 0.4168; WTres_1: 3.9632; WTres_2: -4.0867; 
Epoch 11; Avg_val_loss: 1.0503; WTcoef_1: 0.6824; WTres_1: 2.9029; WTres_2: -4.4585; 
Epoch 21; Avg_val_loss: 1.0456; WTcoef_1: 0.9153; WTres_1: 2.6524; WTres_2: -4.6301;